<h1><strong>Run these commands the first time you open this project on your computer.
    <br>You do NOT need to run these commands again -- ever, after the first time.</strong></h1>
<p>Without these commands, running the webcam code will output an error. These commands fix the issue.
<br>You only need to run these commands once. You can exit Jupyter Lab and come back without having to run these commands again.</p>

In [1]:
# run this command the first time you open this notebook
!pip uninstall opencv-python -y

Found existing installation: opencv-python 4.5.4.60
Uninstalling opencv-python-4.5.4.60:
  Successfully uninstalled opencv-python-4.5.4.60


In [7]:
# run this command the first time you open this notebook, after uninstalling
!pip install opencv-python

You should consider upgrading via the 'c:\users\guaco\downloads\ml-project (1)\.venv\scripts\python.exe -m pip install --upgrade pip' command.


<h1><strong>Run each command to load the program.</strong></h1>

In [8]:
# you do not need to run the commands on top if you have already did so the first time
# you may now run the rest of the commands
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

In [9]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [12]:
pip install --upgrade.pip

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\users\guaco\downloads\ml-project (1)\.venv\scripts\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\users\guaco\downloads\ml-project (1)\.venv\scripts\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\users\guaco\downloads\ml-project (1)\.venv\scripts\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\users\guaco\downloads\ml-project (1)\.venv\scripts\python.exe -m pip install [options] [-e] <local project path> ...
  c:\users\guaco\downloads\ml-project (1)\.venv\scripts\python.exe -m pip install [options] <archive url/path> ...

no such option: --upgrade.pip


In [13]:
pip install tensorflow==2.7.0

  Using cached tensorflow-2.7.0-cp37-cp37m-win_amd64.whl (430.8 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [Errno 28] No space left on device

You should consider upgrading via the 'c:\users\guaco\downloads\ml-project (1)\.venv\scripts\python.exe -m pip install --upgrade pip' command.


## Import tensorflow and the object detection modules. 
### This will help set up the model for transfer learning.

In [10]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

ModuleNotFoundError: No module named 'tensorflow'

### Create the path for the model and create the model config 

In [ ]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [ ]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

### Load the pipeline

#### This will help build the detection model for finding gestures. The detection model is what is used when testing out the program in real time using the webcam. Function detect_fn detects that a gesture is being placed on screen by puttting the gesture in a box that then allows for the model to output a prediction.

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-11')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

### Import CV2 and set up the camera for gesture recognition.

In [ ]:
import cv2 
import numpy as np

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [ ]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Wait for previous cell to load before running the next cell.

### This is the main program cell. This will run the program and launch the webcam in order to detect gestures. For every gesture the program recognizes, the model will output a prediction from one of the classes with a measure of confidence. Close the program by pressing 'q'.

In [ ]:
# this command will run the program
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=3,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

<h1><strong>To close the program, press 'q' on your keyboard.<br>The program will freeze.<br>Then, run the next command to close the program.
    <br><br>Or, you may STOP the kernel and then run the next command.</strong></h1>

In [ ]:
# run this command after stopping the kernel to close the program
cap.release()
cv2.destroyAllWindows()